<a href="https://colab.research.google.com/github/yananma/5_programs_per_day/blob/master/02164.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 5.9 含并行连接的网络 ( GoogLeNet )

### 5.9.1 Inception 块

In [0]:
import torch 
from torch import nn, optim 
import torch.nn.functional as F 
import d2l 
import time 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class Inception(nn.Module):
    def __init__(self, in_c, c1, c2, c3, c4):
        super(Inception, self).__init__()
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1, p2, p3, p4), dim=1)

### 5.9.2 GoogLeNet 模型

In [0]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3), 
          nn.ReLU(), 
          nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [0]:
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1), 
          nn.Conv2d(64, 192, kernel_size=3, padding=1), 
          nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [0]:
b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32), 
          Inception(256, 128, (128, 192), (32, 96), 64), 
          nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [0]:
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64), 
          Inception(512, 160, (112, 224), (24, 64), 64), 
          Inception(512, 128, (128, 256), (24, 64), 64), 
          Inception(512, 112, (144, 288), (32, 64), 64), 
          Inception(528, 256, (160, 320), (32, 128), 128), 
          nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [0]:
b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128), 
          Inception(832, 384, (192, 384), (48, 128), 128), 
          d2l.GlobalAvgPool2d())

In [10]:
net = nn.Sequential(b1, b2, b3, b4, b5, d2l.FlattenLayer(), nn.Linear(1024, 10))
X = torch.rand(1, 1, 96, 96)
for blk in net.children():
    X = blk(X)
    print('output shape: ', X.shape)

output shape:  torch.Size([1, 64, 24, 24])
output shape:  torch.Size([1, 192, 12, 12])
output shape:  torch.Size([1, 480, 6, 6])
output shape:  torch.Size([1, 832, 3, 3])
output shape:  torch.Size([1, 1024, 1, 1])
output shape:  torch.Size([1, 1024])
output shape:  torch.Size([1, 10])


### 5.9.3 获取数据和训练模型

In [12]:
batch_size = 128 
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5 
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 1.0029, train acc 0.615, test acc 0.794, time 132.4 sec
epoch 2, loss 0.4163, train acc 0.846, test acc 0.839, time 132.3 sec
epoch 3, loss 0.3385, train acc 0.874, test acc 0.870, time 132.4 sec
epoch 4, loss 0.3003, train acc 0.889, test acc 0.894, time 132.1 sec
epoch 5, loss 0.2697, train acc 0.900, test acc 0.892, time 132.6 sec
